In [2]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset, Subset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from itertools import product
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import lightning as L
import torch.nn.functional as F


In [ ]:
def plot_err_curve(result):

    plt.subplots(1,2,1)
    #accuracy plot
    plt.plot(models_train_acc[i][0:b_ep], label='train_acc') 
    plt.plot(models_val_acc[i][0:b_ep], label='valid_acc', linestyle='dashed')       
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend(loc = 'lower right')

    #Loss plot
    plt.plot(models_train_mse[i][0:b_ep], label='Loss') 
    plt.plot(models_val_mse[i][0:b_ep], label='val_loss', linestyle='dashed')       
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend(loc = 'upper right')

    plt.show()
    

# Pytorch Litghining MLP

In [3]:
class MyDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [25]:
class MLP_Light_NN(L.LightningModule):
    def __init__(self,  input_size, hidden_size, output_size, num_h_layers, f_act, lr, momentum, weight_decay):
        super().__init__()
        self.layers = nn.ModuleList([nn.Linear(input_size, hidden_size)])
        self.func=f_act()
        # Define the hidden layers
        for i in range(1, num_h_layers):
            self.layers.append(nn.Linear(hidden_size, hidden_size))
            
        self.layers.append(nn.Linear(hidden_size, output_size))
        
        self.lr=lr
        self.momentum=momentum
        self.weight_decay=weight_decay

    def forward(self, x):
        for layer in self.layers[:-1]:
            x = self.func(layer(x))
        
        out=self.layers[-1]
        x=F.sigmoid(out(x))
        return x

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        x, y = batch
        x_hat = self(x)
        y = y.reshape(-1, 1)
        loss = F.mse_loss(x_hat, y)
        self.log("tr_loss", loss, prog_bar=False,on_epoch=True, on_step=False)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.parameters(), lr=self.lr,momentum=self.momentum,weight_decay=self.weight_decay)
        return optimizer
    
    def validation_step(self, batch, batch_idx):
        # this is the validation loop
        x, y = batch
        x_hat = self(x)
        y = y.reshape(-1, 1)
        loss = F.mse_loss(x_hat, y)
        self.log("val_loss", loss, prog_bar=False,on_epoch=True, on_step=False)
        return loss


In [26]:
def training_NN(tr_data,tr_labels,dataset,params,param_grid):
    param_dict = dict(zip(param_grid.keys(), params))

    val_acc_1_fold=[]
    tr_acc_1_fold=[]

    for tr_idx, val_idx in StratifiedKFold(n_splits=5).split(tr_data,tr_labels):

        tr_dataset = Subset(dataset, tr_idx)
        val_dataset = Subset(dataset, val_idx)
        tr_loader = DataLoader(tr_dataset, batch_size=None, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=None, shuffle=False)

        #early_stopping = EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=True, min_delta=0.0001)
        trainer = L.Trainer(max_epochs=500,enable_progress_bar=False)
        model=MLP_Light_NN(input_size=tr_data.shape[1], output_size=1, **param_dict)
        trainer.fit(model, tr_loader, val_loader)

        #val_acc_1_fold.append(trainer.callback_metrics['val_acc'])
        #tr_acc_1_fold.append(trainer.callback_metrics['tr_acc'])

    #mean_val_acc = np.mean(val_acc_1_fold)
    #mean_tr_acc = np.mean(tr_acc_1_fold)

    return 0#mean_tr_acc,mean_val_acc

In [27]:
path=r'/home/ludovico/ML-project/data/monks-1'
monk1_train = pd.read_csv(path+'.train', header=None, delim_whitespace=True, dtype=str)
monk1_test = pd.read_csv(path+'.test', header=None, delim_whitespace=True, dtype=str)

tr_labels=monk1_train[0].astype(np.float32)
tr_data=monk1_train[monk1_train.columns[1:-1]]#.astype(np.float32)

test_labels=monk1_test[0].astype(np.float32)
test_data=monk1_test[monk1_train.columns[1:-1]]#.astype(np.float32)

tr_data=pd.get_dummies(tr_data).astype(np.float32)
test_data=pd.get_dummies(test_data).astype(np.float32)

tr_data=torch.tensor(tr_data.values)
test_data=torch.tensor(test_data.values)
tr_labels=torch.tensor(tr_labels.values)
test_labels=torch.tensor(test_labels.values)


In [ ]:
dataset = MyDataset(tr_data, tr_labels)
dataloader = DataLoader(dataset, shuffle=True) 

print(MLP_Light_NN(17,8,1,1,nn.ReLU,1,0.1,0).forward(tr_data[0]))

param_grid = {
    'num_h_layers': [1],
    'hidden_size': [8],
    'f_act': [nn.ReLU],
    'lr': [0.1],
    'momentum': [0.1], 
    'weight_decay': [0], 
}


# Generate all possible combinations of hyperparameters
param_combinations = product(*param_grid.values())
    

for params in param_combinations:
    print(training_NN(tr_data,tr_labels, dataset, params,param_grid))


